In [270]:
import pandas as pd
import requests
import json 
import http.client, urllib.request, urllib.parse, urllib.error, base64
import http
import io
import newspaper
from multiprocessing import Pool
from textblob import TextBlob
import time 

In [245]:
api_key = "bbc19a8158e147b79f964c7155272cb4"

In [271]:
t = time.time()
df = make_news_requ(sample_req)

print(t-time.time())


text finish
-33.695647954940796


In [262]:
sample_req = {'tree': "everything", "sources": [], 'sortBy': 'popularity', 'keywords': ['trump'],"date": "", "to":''}
###final function which returns the entire dataframe with atributes
def make_news_requ(input_dict):
    base = 'https://newsapi.org/v2/'
    tree = input_dict['tree']+ "?"
    ###############
    keywords = "q="+ ',+'.join(input_dict['keywords'])
    date = 'from=' + input_dict['date']
    to = "to=" + input_dict['to']
    sortBy = 'sortBy=popularity'
    sources = "sources=" + ','.join(input_dict['sources'])
    api_key = "apiKey=bbc19a8158e147b79f964c7155272cb4"
    
    add_on = ""
    
    for dic, ins in [["sources", sources],["sortBy", sortBy],["date",date],['to',to]]:
        if len(input_dict[dic]) !=0:
            add_on += "&"+ins
                               
    url = base+tree+keywords + add_on + "&" +  api_key
                                    
    response = requests.get(url)
    jsonified = json.loads(response.text)
    
    atr = jsonified['articles']
    df = pd.DataFrame(columns = atr[0].keys())
    #df = pd.DataFrame(atr[0])
    for x in atr[1:]:
        x['source'] = x['source']['name']
        x['bias'] = article_text(x['url'])
        df = df.append(x, ignore_index=True)
    print("text finish")
    df['e_at_pic'] = df['urlToImage'].apply(func = emotion_fin)#pandas more inefficient use multithreading
    return df

In [275]:
###creates a function which returns the emotions displayed on a picutre to a corresbonding art.
def keywithmaxval(d): 
     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]

sattributes = ['age','glasses','smile', 'gender']
maxattributes = ['emotion','facialHair']

def add_to_df(instan):
    if len(instan)  == 0:
        return "no faces"
    else:
        facial_tempdic = {}

        for attri in sattributes: 
            facial_tempdic[attri] =  instan[0]['faceAttributes'][attri]
        for attri in maxattributes:
            stats = instan[0]['faceAttributes'][attri]
            facial_tempdic[attri] = keywithmaxval(stats) + " "+str(stats[keywithmaxval(stats)])
        return facial_tempdic
    
def emotion_to_image(filename):
    uri_base = 'https://westcentralus.api.cognitive.microsoft.com'
    subscription_key = '4b5604997f8747448756700dfac8a51d'

    headers = {
    'Content-Type': 'application/octet-stream',
    'Ocp-Apim-Subscription-Key': subscription_key,}

    params = {
   'returnFaceId': 'true', 'Adult': 'true',
   'returnFaceLandmarks': 'false',
   'returnFaceAttributes': 'age,gender,smile,facialHair,glasses,emotion,hair,makeup'}

    path_to_face_api = '/face/v1.0/detect'
    
    with urllib.request.urlopen(filename) as url:
        img_data = io.BytesIO(url.read())
    try:
        response = requests.post(uri_base + path_to_face_api,
                                data=img_data,
                                headers=headers,
                                params=params)
        parsed = response.json()
        return(parsed)
       
    except Exception as e:
        return([])
    
def emotion_fin(link):
    try:
        result = add_to_df(emotion_to_image(link))
    except:
        result = []
    return result

In [263]:
###gives a sentiment for url
def article_text(article_url):
   try:
       article = newspaper.Article(url=article_url, language='en')
       article.download()
       article.parse()
       textb = TextBlob(article.text) 
       return textb.sentiment
   except newspaper.ArticleException:
       print("URL: %s not found" % article_url)
   return ""

# def article_dict(urls):
#    pool = Pool(len(urls))
#    texts = pool.map(article_text, urls)
#    if not len(texts) == len(urls):
#        raise RuntimeError("article_dict(): Array sizes do not match !")
#    url_text_dict = {}
#    for x in range(0, len(texts)):
#        url_text_dict[urls[x]] = texts[x]
#    return url_text_dict


In [214]:
sample = {"documents":[{'id': 1, 'text': 'bad ugly shit'},{'id':2,"text": "good well awesome fabulous"}]}

In [242]:
haha = TextBlob("Trump could be better as his job")

In [261]:
article_text('https://www.theroot.com/5-ways-white-people-can-fight-white-supremacy-1819286547')

Sentiment(polarity=0.07508672100468976, subjectivity=0.45208116319444436)

In [276]:
emotion_to_image("https://static.politico.com/dims4/default/fd0ac72/2147483647/resize/1160x%3E/quality/90/?url=https%3A%2F%2Fstatic.politico.com%2F2d%2F1c%2F7cf287c64bffbe597523c0e9cf69%2F171121-trump-gty-1160.jpg")

{'error': {'code': 'Unspecified',
  'message': 'Access denied due to invalid subscription key. Make sure you are subscribed to an API you are trying to call and provide the right key.'}}